# This notebook tries to improve how well the model describes the reality by adjusting it's weights

In [1]:
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('data/seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('data/qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('data/driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('data/constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('data/enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [3]:
entries = []
results = []
cleaner = F1DataCleaner(seasonsData, qualiResultsData, driversData, constructorsData, enginesData)

#Constants we can change

cleaner.k_engine_change = 0.0145
cleaner.k_const_change = 0.240
cleaner.k_driver_change = 0.19
cleaner.k_const_impact = 0.80
cleaner.k_eng_impact = (1 - cleaner.k_const_impact)
cleaner.k_driver_impact = 0.02
        
cleaner.k_rookie_pwr = 0.40
#cleaner.k_rookie_variance = 5
cleaner.k_race_regress_exp = 0.87  #TODO needs to change!
#cleaner.k_variance_multiplier_end = 1.5

cleaner.k_eng_regress = 1.04
cleaner.k_const_regress = 0.60
cleaner.k_driver_regress = 0.67

cleaner.constructDataset(entries, results)
#print(entries[-10:])
#print(results[-10:])
X = np.array(entries)
y = results

forest = RandomForestRegressor(random_state=0, n_estimators=100)
forest.fit(X, y)
print(forest.feature_importances_)
#Rates based on test data!
print(forest.score(X, y))

[0.15248337 0.58564437 0.09507341 0.0485429  0.0517972  0.06645875]
0.9416018084532017


In [4]:
#Linear regression
reg = LinearRegression().fit(X, y)
print("R-score: " + str(reg.score(X, y)))
print(reg.coef_)

R-score: 0.5372474260676956
[0.52667928 0.82999174 0.67659264 0.1043197  0.20368626 2.23385799]


In [5]:
#Random forest regression

forest = RandomForestRegressor(max_depth=5, random_state=0,
                             n_estimators=100)
forest.fit(X, y)
print(forest.feature_importances_)
#Rates based on test data!
print(forest.score(X, y))

[0.11048827 0.82029603 0.034428   0.00399929 0.00579697 0.02499145]
0.6302162641292898


# Just for testing!

In [6]:
newDrivers = json.load(open('data/newDrivers.json'))["drivers"]
newDrivers = {int(did): cid for did, cid in newDrivers.items()}

#Manual changes
#cleaner.drivers[8].constructor = cleaner.constructors[15] #Kimi
#cleaner.drivers[841].constructor = cleaner.constructors[15] #Gio
#cleaner.drivers[844].constructor = cleaner.constructors[6] #Leclerc
#cleaner.drivers[1000] = Driver("Lando Norris", cleaner.constructors[1]) #Lando
#cleaner.drivers[832].constructor = cleaner.constructors[1] #Sainz
#cleaner.drivers[840].constructor = cleaner.constructors[10] #Stroll
#cleaner.drivers[842].constructor = cleaner.constructors[9] #Gasly
#cleaner.drivers[817].constructor = cleaner.constructors[4] #Ricciardo
#cleaner.drivers[1001] = Driver("Alexander Albon", cleaner.constructors[5]) #Albon
#cleaner.drivers[826].constructor = cleaner.constructors[5] #Kvyat
#cleaner.drivers[1002] = Driver("George Russell", cleaner.constructors[3]) #Russell
#cleaner.drivers[9].constructor = cleaner.constructors[3] #Kubica

#cleaner.drivers[1000].pwr = cleaner.k_rookie_pwr
#cleaner.drivers[1001].pwr = cleaner.k_rookie_pwr
#cleaner.drivers[1002].pwr = cleaner.k_rookie_pwr

driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = cleaner.drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = cleaner.drivers[int(did)].constructor.name
print(driversToWrite)
with open('../F1PredictWeb/public/driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

{8: {'name': 'Kimi Räikkönen', 'constructor': 'Alfa Romeo'}, 841: {'name': 'Antonio Giovinazzi', 'constructor': 'Alfa Romeo'}, 20: {'name': 'Sebastian Vettel', 'constructor': 'Ferrari'}, 844: {'name': 'Charles Leclerc', 'constructor': 'Ferrari'}, 154: {'name': 'Romain Grosjean', 'constructor': 'Haas F1 Team'}, 825: {'name': 'Kevin Magnussen', 'constructor': 'Haas F1 Team'}, 846: {'name': 'Lando Norris', 'constructor': 'McLaren'}, 832: {'name': 'Carlos Sainz', 'constructor': 'McLaren'}, 1: {'name': 'Lewis Hamilton', 'constructor': 'Mercedes'}, 822: {'name': 'Valtteri Bottas', 'constructor': 'Mercedes'}, 815: {'name': 'Sergio Pérez', 'constructor': 'Racing Point'}, 840: {'name': 'Lance Stroll', 'constructor': 'Racing Point'}, 842: {'name': 'Pierre Gasly', 'constructor': 'Red Bull'}, 830: {'name': 'Max Verstappen', 'constructor': 'Red Bull'}, 817: {'name': 'Daniel Ricciardo', 'constructor': 'Renault'}, 807: {'name': 'Nico Hülkenberg', 'constructor': 'Renault'}, 848: {'name': 'Alexander Al

In [7]:
circuit = 6
circuitName = "Monaco"

with open('../F1PredictWeb/public/circuitName.txt', 'w') as fp:
    fp.write(circuitName);

In [8]:
predictedEntrants = []

for did, cid in newDrivers.items():
    print(cleaner.drivers[did].name + ": " + str(cleaner.drivers[did].pwr) + " -- " + str(cleaner.drivers[did].constructor.pwr))
    if circuit not in cleaner.drivers[did].trackpwr:
        cleaner.drivers[did].trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.trackpwr:
        cleaner.drivers[did].constructor.trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.engine.trackpwr:
        cleaner.drivers[did].constructor.engine.trackpwr[circuit] = 0 #TODO maybe change defaults
    
    entry = [
        cleaner.drivers[did].pwr,
        cleaner.drivers[did].constructor.pwr, 
        cleaner.drivers[did].constructor.engine.pwr,
        cleaner.drivers[did].trackpwr[circuit],
        cleaner.drivers[did].constructor.trackpwr[circuit],
        cleaner.drivers[did].constructor.engine.trackpwr[circuit]
    ]
    predictedEntrants.append(entry)

Kimi Räikkönen: 0.0044700537878007394 -- 0.10951724991614968
Antonio Giovinazzi: 0.20969678174057774 -- 0.10951724991614968
Sebastian Vettel: -0.12643736968187064 -- -0.45486740701213374
Charles Leclerc: -0.12156005366633975 -- -0.45486740701213374
Romain Grosjean: -0.1019951468506386 -- -0.08098352768157459
Kevin Magnussen: 0.01428857042245615 -- -0.08098352768157459
Lando Norris: 0.04549458276528203 -- 0.09237614026570634
Carlos Sainz: 0.019142550143476142 -- 0.09237614026570634
Lewis Hamilton: -0.2535435988637111 -- -0.44221035892911553
Valtteri Bottas: -0.29090770860553783 -- -0.44221035892911553
Sergio Pérez: -0.035076679729627365 -- 0.26551354609274036
Lance Stroll: 0.24364470221147863 -- 0.26551354609274036
Pierre Gasly: 0.11212815510485762 -- -0.36418406875225284
Max Verstappen: -0.16511566092799196 -- -0.36418406875225284
Daniel Ricciardo: -0.03634531417425098 -- 0.15397759903101893
Nico Hülkenberg: 0.16995588800667746 -- 0.15397759903101893
Alexander Albon: 0.1063662452738008

In [9]:
#Linear regression predict
predictedResults = reg.predict(np.array(predictedEntrants))
print(predictedResults)

[ 0.20741436  0.31890563 -0.41009203 -0.40830234 -0.03781001  0.03168384
  0.00636527 -0.02143347 -0.65130528 -0.66089451 -0.01356144  0.15951507
 -0.18402743 -0.32994826  0.05626652  0.18513168  0.16583371  0.06623345
  0.86743707  0.92467753]


In [10]:
#Forest predict
forestResults = forest.predict(np.array(predictedEntrants))
print(forestResults)

[ 0.12705847  0.27083898 -0.43249903 -0.43007875 -0.12058732 -0.07811074
  0.16814876  0.07222324 -0.60481357 -0.60481357  0.00769483  0.21840141
 -0.37174265 -0.41620242  0.05902698  0.30375842  0.03140118  0.03078103
  0.80623562  0.69305622]


In [11]:
driverResults = {} # {did: {position: amount}}
for index, (did, cid) in enumerate(newDrivers.items()):
    print("{0} ({1}): {2}".format(cleaner.drivers[int(did)].name, cleaner.drivers[int(did)].constructor.name, forestResults[index]))
    newDrivers[did] = forestResults[index]
    driverResults[int(did)] = {}
    
for i in range(1000):
    scoreList = predictQualiResults(circuit, newDrivers)
    for i, drivRes in enumerate(scoreList):
        if i not in driverResults[drivRes[0]]:
            driverResults[drivRes[0]][i] = 0
        driverResults[drivRes[0]][i] += 1
        
for did, res in driverResults.items():
    print("{0} ({1}):".format(cleaner.drivers[int(did)].name, cleaner.drivers[int(did)].constructor.name))
    for pos, amount in sorted(res.items(), key=lambda posAmount: posAmount[0]):
        print("\t{0}: {1} %".format(pos + 1, amount / 10.0))
        
with open('../F1PredictWeb/public/predictions.json', 'w') as fp:
    json.dump(driverResults, fp)

Kimi Räikkönen (Alfa Romeo): 0.12705847000540665
Antonio Giovinazzi (Alfa Romeo): 0.2708389791122876
Sebastian Vettel (Ferrari): -0.43249902923413186
Charles Leclerc (Ferrari): -0.43007874645366556
Romain Grosjean (Haas F1 Team): -0.1205873203260693
Kevin Magnussen (Haas F1 Team): -0.07811073917324234
Lando Norris (McLaren): 0.16814876147448615
Carlos Sainz (McLaren): 0.07222324096334636
Lewis Hamilton (Mercedes): -0.6048135676383841
Valtteri Bottas (Mercedes): -0.6048135676383841
Sergio Pérez (Racing Point): 0.007694830695348378
Lance Stroll (Racing Point): 0.2184014134728162
Pierre Gasly (Red Bull): -0.3717426531998484
Max Verstappen (Red Bull): -0.4162024208779953
Daniel Ricciardo (Renault): 0.059026977201972065
Nico Hülkenberg (Renault): 0.30375841509311485
Alexander Albon (Toro Rosso): 0.03140117727349035
Daniil Kvyat (Toro Rosso): 0.03078103185596603
George Russell (Williams): 0.80623562403292
Robert Kubica (Williams): 0.6930562164162661
Kimi Räikkönen (Alfa Romeo):
	7: 0.3 %
	8:

-0.3754633047458623
-0.04901705829058218
